Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

Want to iterate through the data rather than adding manually (358 bands)

Then need to split into X (all the bands) and y (PFT)

When complete, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [1]:
import pandas as pd
import geopandas
cross_lochs = pd.read_csv('cross_lochs_sample_for_rf2.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf


/home/rzw/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
0,617,0.000951,0.005150,0.005988,0.006720,0.007361,0.010606,0.012082,0.014234,0.015797,...,0.097626,0.096350,0.095506,0.098056,0.096859,618,POINT (444764.308 6472590.655),0.000951,calluna_mix,4
1,822,0.002991,0.004075,0.004819,0.005989,0.007388,0.009729,0.012035,0.013280,0.016029,...,0.091884,0.090096,0.090060,0.089917,0.090350,823,POINT (444774.308 6472585.655),0.002991,calluna_mix,5
2,1230,0.002812,0.003526,0.005374,0.006186,0.007536,0.009726,0.011938,0.013003,0.016773,...,0.087527,0.090578,0.089190,0.087724,0.090253,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix,7
3,1841,0.002776,0.004215,0.004849,0.006305,0.008160,0.010193,0.012259,0.014886,0.015938,...,0.093998,0.096585,0.099090,0.097692,0.098034,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix,10
4,2452,0.003292,0.003483,0.003686,0.005693,0.007814,0.009010,0.012300,0.013145,0.015174,...,0.094432,0.094272,0.097161,0.097323,0.095702,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,19455,0.000811,0.001748,0.002514,0.003086,0.002629,0.003547,0.005152,0.006167,0.008431,...,0.010550,0.012407,0.010226,0.010765,0.010614,19456,POINT (445574.308 6472130.655),0.000811,water,96
176,21481,0.000730,0.001129,0.001588,0.002542,0.002014,0.003243,0.004675,0.004775,0.006566,...,0.005632,0.006227,0.007418,0.005301,0.004491,21482,POINT (445554.308 6472080.655),0.000730,water,106
177,22294,0.000828,0.001481,0.002596,0.002506,0.002702,0.003921,0.005196,0.006644,0.008155,...,0.010491,0.014097,0.011939,0.011159,0.008924,22295,POINT (445559.308 6472060.655),0.000828,water,110
178,41439,0.000852,0.001206,0.003087,0.003662,0.005048,0.007325,0.009386,0.011490,0.013876,...,0.080829,0.080322,0.081170,0.080854,0.080922,41440,POINT (444859.308 6471585.655),0.000852,calluna_mix,205


In [3]:
# cross_lochs_vis = cross_lochs_gdf[cross_lochs_gdf.columns[7:66],[359:363]] # issues with the second set of square brackets

cross_lochs_vis_bands = cross_lochs_gdf.iloc[:,6:66]
cross_lochs_RE_bands = cross_lochs_gdf.iloc[:,62:76]
cross_lochs_NIR_bands = cross_lochs_gdf.iloc[:,66:186]
cross_lochs_SWIR_bands = cross_lochs_gdf.iloc[:,186:359]

cross_lochs_general = cross_lochs_gdf.iloc[:,359:363]

cross_lochs_vis = pd.concat([cross_lochs_vis_bands, cross_lochs_general], axis=1)
cross_lochs_RE = pd.concat([cross_lochs_RE_bands, cross_lochs_general], axis=1)
cross_lochs_NIR = pd.concat([cross_lochs_NIR_bands, cross_lochs_general], axis = 1)
cross_lochs_SWIR = pd.concat([cross_lochs_SWIR_bands, cross_lochs_general], axis = 1)


cross_lochs_SWIR.head()

,band186,band187,band188,band189,band190,band191,band192,band193,band194,band195,...,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT
0,0.410717,0.401726,0.395631,0.395313,0.390486,0.380488,0.409504,0.582088,0.540145,0.258842,...,0.096551,0.097626,0.096350,0.095506,0.098056,0.096859,618,POINT (444764.308 6472590.655),0.000951,calluna_mix
1,0.383370,0.376514,0.373993,0.375042,0.365768,0.359697,0.382855,0.560134,0.505520,0.260660,...,0.089470,0.091884,0.090096,0.090060,0.089917,0.090350,823,POINT (444774.308 6472585.655),0.002991,calluna_mix
2,0.373384,0.370922,0.364682,0.363427,0.359305,0.355752,0.372333,0.568716,0.483928,0.267694,...,0.089292,0.087527,0.090578,0.089190,0.087724,0.090253,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix
3,0.378747,0.375747,0.368290,0.368852,0.361958,0.358295,0.379347,0.562003,0.504111,0.254960,...,0.098643,0.093998,0.096585,0.099090,0.097692,0.098034,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix
4,0.414651,0.409048,0.404458,0.405261,0.400736,0.392717,0.415928,0.609917,0.513328,0.247289,...,0.095850,0.094432,0.094272,0.097161,0.097323,0.095702,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix


In [4]:
from sklearn.model_selection import train_test_split

train_shrub_sphagnum, test_shrub_sphagnum = train_test_split(cross_lochs_SWIR[cross_lochs_SWIR['PFT'].str.contains('shrub_sphagnum')], test_size=0.3)
train_water, test_water = train_test_split(cross_lochs_SWIR[cross_lochs_SWIR['PFT'].str.contains('water')], test_size=0.3)
train_sphagnum_r, test_sphagnum_r = train_test_split(cross_lochs_SWIR[cross_lochs_SWIR['PFT'].str.contains('sphagnum_r')], test_size=0.3)
train_pool_bogbean, test_pool_bogbean = train_test_split(cross_lochs_SWIR[cross_lochs_SWIR['PFT'].str.contains('pool_bogbean')], test_size=0.3)
train_calluna_mix, test_calluna_mix = train_test_split(cross_lochs_SWIR[cross_lochs_SWIR['PFT'].str.contains('calluna_mix')], test_size=0.3)
train_rushes_sedges, test_rushes_sedges = train_test_split(cross_lochs_SWIR[cross_lochs_SWIR['PFT'].str.contains('rushes_sedges')], test_size=0.3)

train_frames = [train_shrub_sphagnum, train_water, train_sphagnum_r, train_pool_bogbean, train_calluna_mix, train_rushes_sedges]
test_frames = [test_shrub_sphagnum, test_water, test_sphagnum_r, test_pool_bogbean, test_calluna_mix, test_rushes_sedges]


train = pd.concat(train_frames)
test = pd.concat(test_frames)


train.head()


,band186,band187,band188,band189,band190,band191,band192,band193,band194,band195,...,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT
94,0.366963,0.360519,0.357759,0.363252,0.348076,0.342836,0.371539,0.529494,0.441613,0.225229,...,0.086378,0.087599,0.085849,0.087763,0.083878,0.086655,13278,POINT (445134.308 6472280.655),0.001527,shrub_sphagnum
96,0.349430,0.346014,0.342165,0.339988,0.327805,0.327558,0.344730,0.491997,0.401587,0.256977,...,0.082353,0.080677,0.082388,0.082734,0.079711,0.078884,17352,POINT (445204.308 6472180.655),0.001153,shrub_sphagnum
111,0.402522,0.399672,0.393266,0.395141,0.381422,0.377944,0.399590,0.569016,0.490962,0.209691,...,0.094573,0.097958,0.098143,0.096620,0.097980,0.097324,33755,POINT (445004.308 6471775.655),0.000855,shrub_sphagnum
90,0.356433,0.352759,0.346392,0.346446,0.332319,0.330989,0.355359,0.487946,0.390859,0.207749,...,0.077513,0.081846,0.078069,0.081465,0.081287,0.079147,9623,POINT (445129.308 6472370.655),0.001516,shrub_sphagnum
95,0.374674,0.369240,0.365344,0.360695,0.352276,0.351648,0.371416,0.511937,0.428984,0.225710,...,0.085597,0.087433,0.087913,0.090433,0.087537,0.085878,13870,POINT (445049.308 6472265.655),0.001928,shrub_sphagnum


In [5]:
x_train = train[train.columns[0:173]]
y_train = train[train.columns[176]]

x_test = test[test.columns[0:173]]
y_test = test[test.columns[176]]

x_test.head(2)

,band186,band187,band188,band189,band190,band191,band192,band193,band194,band195,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
92,0.396136,0.391899,0.387952,0.387859,0.378852,0.371866,0.397084,0.547597,0.517182,0.204364,...,0.091438,0.093438,0.093616,0.094628,0.092816,0.095677,0.091634,0.094247,0.093502,0.091627
117,0.302441,0.301741,0.294314,0.294596,0.288723,0.283833,0.296331,0.443264,0.400331,0.315255,...,0.068315,0.068021,0.068702,0.070609,0.066388,0.067452,0.066715,0.067169,0.068227,0.065162


In [6]:
from sklearn import dummy

dc = dummy.DummyClassifier()
dc.fit(x_train, y_train)
dc_score = dc.score(x_test, y_test)

print("Baseline Accuracy:", dc_score)


Baseline Accuracy: 0.16666666666666666


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=2, random_state=42)
# fit to the training data
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))

# print(clf.predict(x_test))


# predict using the model
clf_train_pred = clf.predict(x_train)
clf_pred = clf.predict(x_test)

# append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# test the performance of the model
scores = cross_val_score(clf, x_train, y_train, cv=3)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.6666666666666666
0.61 accuracy with a standard deviation of 0.07


In [8]:
test_score = cross_val_score(clf, x_test, y_test, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (test_score.mean(), test_score.std()))

0.59 accuracy with a standard deviation of 0.07


In [9]:
model_results = test[test.columns[173:176]]

model_results.head()

,POINTID,geometry,GRID_CODE
92,11852,POINT (445109.308 6472315.655),0.001311
117,41717,POINT (445229.308 6471580.655),0.000747
98,18770,POINT (445189.308 6472145.655),0.000968
97,18563,POINT (445169.308 6472150.655),0.001223
115,38653,POINT (445134.308 6471655.655),0.000281


In [10]:
train_focused = train[train.columns[173:176]]

train_focused.head()

,POINTID,geometry,GRID_CODE
94,13278,POINT (445134.308 6472280.655),0.001527
96,17352,POINT (445204.308 6472180.655),0.001153
111,33755,POINT (445004.308 6471775.655),0.000855
90,9623,POINT (445129.308 6472370.655),0.001516
95,13870,POINT (445049.308 6472265.655),0.001928


In [11]:
model_results.to_file("cross_lochs_results_sample_SWIR_gdf.shp")

In [12]:
train_focused.to_file("cross_lochs_train_sample_SWIR_gdf.shp")

In [13]:
model_results.to_csv("cross_lochs_results_sample_SWIR_gdf.csv")
train_focused.to_csv("cross_lochs_train_sample_SWIR_gdf.csv")

In [14]:
# now want to predict for the whole dataset using the random forest
cross_lochs = pd.read_csv('../cross_lochs_gdf.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT
0,0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,...,0.074587,0.072792,0.075150,0.074088,0.074698,0.072248,1,POINT (444724.308 6472605.655),0.001099,NaN
1,1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,...,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113,2,POINT (444729.308 6472605.655),0.003365,NaN
2,2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,...,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678,3,POINT (444734.308 6472605.655),0.002603,NaN
3,3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,...,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883,4,POINT (444739.308 6472605.655),0.002381,NaN
4,4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,...,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215,5,POINT (444744.308 6472605.655),0.002342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42016,42016,0.001845,0.002345,0.004714,0.005430,0.006393,0.008860,0.010772,0.012449,0.014887,...,0.077407,0.076553,0.078615,0.078627,0.078251,0.077265,42017,POINT (445714.308 6471575.655),0.001845,NaN
42017,42017,0.002455,0.002271,0.004029,0.005480,0.006396,0.008863,0.011025,0.012685,0.014688,...,0.073803,0.074018,0.074871,0.075158,0.073604,0.075263,42018,POINT (445719.308 6471575.655),0.002455,NaN
42018,42018,0.002387,0.002279,0.004424,0.005426,0.006352,0.008929,0.010853,0.012708,0.014964,...,0.082753,0.083324,0.083870,0.084588,0.081337,0.082178,42019,POINT (445724.308 6471575.655),0.002387,NaN
42019,42019,0.002777,0.002263,0.004564,0.005324,0.006339,0.008972,0.011173,0.012952,0.015367,...,0.094481,0.092089,0.094319,0.095786,0.090843,0.091052,42020,POINT (445729.308 6471575.655),0.002777,NaN


In [15]:
x = cross_lochs_gdf[cross_lochs_gdf.columns[186:359]]

x.head()

,band186,band187,band188,band189,band190,band191,band192,band193,band194,band195,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
0,0.451655,0.442290,0.431777,0.430757,0.415500,0.407832,0.428181,0.628633,0.570676,0.236753,...,0.077946,0.077413,0.077153,0.078934,0.074587,0.072792,0.075150,0.074088,0.074698,0.072248
1,0.447854,0.441114,0.429510,0.427842,0.415079,0.404171,0.424823,0.623167,0.536215,0.247038,...,0.075149,0.073557,0.071610,0.071727,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113
2,0.452372,0.452207,0.441673,0.435738,0.430362,0.411139,0.436695,0.643019,0.570591,0.244219,...,0.088601,0.087386,0.087973,0.089698,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678
3,0.475654,0.468367,0.460588,0.458941,0.448625,0.441053,0.471728,0.695727,0.619741,0.216395,...,0.105301,0.103799,0.104660,0.107808,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883
4,0.466977,0.459697,0.452623,0.450112,0.436777,0.427441,0.457334,0.664155,0.564976,0.233377,...,0.095351,0.096855,0.094500,0.093532,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215


In [16]:
clf_pred_full = clf.predict(x)

clf_pred_full

array(['sphagnum_r', 'sphagnum_r', 'calluna_mix', ..., 'calluna_mix',
       'calluna_mix', 'calluna_mix'], dtype=object)

In [17]:
model_results_full = cross_lochs_gdf[cross_lochs_gdf.columns[360:362]]

model_results_full = model_results_full.assign(clf_pred_full = clf_pred_full)

model_results_full.head()

,geometry,GRID_CODE,clf_pred_full
0,POINT (444724.308 6472605.655),0.001099,sphagnum_r
1,POINT (444729.308 6472605.655),0.003365,sphagnum_r
2,POINT (444734.308 6472605.655),0.002603,calluna_mix
3,POINT (444739.308 6472605.655),0.002381,calluna_mix
4,POINT (444744.308 6472605.655),0.002342,calluna_mix


In [18]:
model_results_full.to_file("cross_lochs_results_sample_full_SWIR_gdf.shp")

model_results_full.to_csv("cross_lochs_results_sample_full_SWIR_gdf.csv")


/tmp/ipykernel_3697/3253597770.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  model_results_full.to_file("cross_lochs_results_sample_full_SWIR_gdf.shp")
